In [ ]:
# The intention of this notebook is to implement the 12 open-sourced tools from the paper
# Augmenting large language models with chemistry tools
# https://github.com/ur-whitelab/chemcrow-public.git

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
# !pip install -r ../requirements.txt
# !pip install git+https://github.com/blackadad/paper-scraper.git

In [23]:
import os
from langchain import SerpAPIWrapper
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

#### 1. General - WebSearch

In [4]:
def web_search(keywords, search_engine="google"):
    try:
        return SerpAPIWrapper(
            serpapi_api_key=os.getenv("SERP_API_KEY"), search_engine=search_engine
        ).run(keywords)
    except Exception as e:
        print(e)
        return "No results, try another search"

In [5]:
class WebSearch(BaseTool):
    name = "WebSearch"
    description = (
        "Input a specific question, returns an answer from literature search. "
        "Do not mention any specific molecule names, but use more general features to formulate your questions."
    )
    def _run(self, query: str) -> str:
        return web_search(query)
    async def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not implemented")

In [6]:
tools = [WebSearch()]
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.1,
)

agent = initialize_agent(
    llm=llm,
    tools=tools,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [6]:
agent.run('what is the melting point of water?')



> Entering new AgentExecutor chain...
I should search for the melting point of water.
Action: WebSearch
Action Input: "melting point of water"
Observation: ["Water is an inorganic compound with the chemical formula H₂O. It is a transparent, tasteless, odorless, and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms.", 'Water type: Compounds.', 'Water density: 997 kg/m³.', 'Water formula: H₂O.', 'Water boiling_point: 212°F (100°C).', 'Water molar_mass: 18.01528 g/mol.', 'Water chem_spider_id: 937.', 'Water classification: Inorganic compound.', 'Water iupac_id: Oxidane, Water.', 'The melting point of water is the temperature at which it changes from solid ice into liquid water. The solid and liquid phase of water are in ...', 'The normal boiling point of water is 100oC because this is the temperature at which the vapor pressure of water is 760 mmHg, or 1 atm. Under normal ...', 'The melting point of wa

'The melting point of water is 0 °C, 32 °F, or 273 K.'

#### 2. General - LitSearch (Literature Search)

In [1]:
# Look at litsearch.py for more examples

#### 3. General - Python REPL

In [17]:
python_repl = PythonREPL()
python_repl.run("print(1+1)")

'2\n'

In [18]:
python_repl.run("""import numpy as np\nprint(np.random.rand(3,3))""")

'[[0.91982698 0.10518415 0.59731295]\n [0.65772039 0.82286631 0.60523723]\n [0.46525457 0.67607608 0.19111699]]\n'

In [19]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [21]:
tools = [repl_tool]
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.1,
)

agent = initialize_agent(
    llm=llm,
    tools=tools,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

agent.run("function to calculate the sum of two numbers")



> Entering new AgentExecutor chain...
I can define a function that takes two numbers as input and returns their sum.
Action: python_repl
Action Input: def sum_numbers(a, b):
                return a + b
Observation: 
Thought:The function has been defined.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I need to test the function to see if it works correctly.
Action: python_repl
Action Input: print(sum_numbers(3, 5))
Observation: 8

Thought:The function seems to be working correctly.
Final Answer: The sum of 3 and 5 is 8.

> Finished chain.


'The sum of 3 and 5 is 8.'

#### 4. General Search - Human Input

In [27]:
llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)
tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [29]:
agent_chain.run("What's my friend Eric's surname?")



> Entering new AgentExecutor chain...
I don't have access to personal information about your friend, so I'll need to ask a human for guidance.
Action: human
Action Input: What is your friend's surname?

What is your friend's surname?

Observation: donk
Thought:I now know my friend's surname is Donk.
Final Answer: Donk

> Finished chain.


'Donk'

#### 